In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [13]:
import pandas as pd

In [14]:
df=pd.read_csv("/home/yadagiri/Downloads/Hc3Finalfeatures (3).csv")

In [15]:
df

,text,avg line length,vocab,word density,stopwords_count,word count,active,passive,punctuation_count,linking_words_count,...,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ,label
0,"['Basically there are many categories of "" Bes...",21.461538,133,47.670251,124,279,13,0,68,36,...,0,24,5,0,0,24,0,0,12,0
1,['salt is good for not dying in car crashes an...,19.318182,219,51.529412,194,425,17,5,65,67,...,2,48,3,0,3,48,0,0,15,0
2,"[""The way it works is that old TV stations got...",32.800000,231,46.951220,200,492,11,4,75,69,...,4,50,16,1,0,50,0,0,13,0
3,"[""You ca n't just go around assassinating the ...",19.181818,133,63.033175,99,211,10,1,46,23,...,1,16,1,0,0,16,0,0,7,0
4,['Wanting to kill the shit out of Germans driv...,26.727273,168,57.142857,139,294,8,3,41,63,...,6,27,4,1,0,27,0,0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48639,"[""It's not uncommon for blood pressure to fluc...",32.800000,92,56.097561,79,164,5,0,19,26,...,0,14,1,0,0,14,0,0,5,1
48640,['There are several possible causes of a painl...,24.250000,105,54.123711,85,194,8,0,46,44,...,0,16,0,0,0,16,0,0,9,1
48641,"[""It is not appropriate for me to recommend a ...",32.800000,88,53.658537,79,164,3,2,24,33,...,0,11,0,0,0,11,0,0,7,1
48642,['It is not uncommon for people with rheumatoi...,28.400000,95,66.901408,65,142,3,2,22,29,...,0,16,3,0,0,16,0,0,9,1


In [16]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import torch.nn as nn

# Load CSV file
df = pd.read_csv("/home/yadagiri/Downloads/Hc3Finalfeatures (3).csv")

# Define columns
text_column = "text"
numerical_columns = ['avg line length', 'vocab', 'word density', 'stopwords_count', 'word count', 'active', 'passive', 'punctuation_count', 'linking_words_count', 'NOUN', 'VERB', 'PUNCT', 'DET', 'PRON', 'PROPN', 'ADJ', 'AUX', 'ADV', 'PART', 'SCONJ', 'NUM', 'X', 'INTJ', 'ADP', 'SYM', 'SPACE', 'CCONJ']
y_label_column = "label"

# Split dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, text_column, numerical_columns, label_column, max_length=128):
        # Initialize data
        self.texts = df[text_column].tolist()  # List of strings
        self.numerical_data = df[numerical_columns].values  # Numpy array of numerical data
        self.labels = df[label_column].values  # Numpy array of labels

        # Convert to PyTorch tensors
        self.numerical_data = torch.tensor(self.numerical_data, dtype=torch.float)
        self.labels = torch.tensor(self.labels, dtype=torch.long)

        # Initialize tokenizer and other settings
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Get the text and numerical data for the given index
        text = self.texts[idx]
        numerical = self.numerical_data[idx]

        # Tokenize text and convert it to PyTorch tensors
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Convert input_ids and attention_mask to LongTensor
        input_ids = encoding['input_ids'].squeeze(0).long()
        attention_mask = encoding['attention_mask'].squeeze(0).long()

        # Combine numerical data with input_ids and attention mask
        combined_input = torch.cat((numerical, input_ids, attention_mask), dim=0)

        # Return input_ids, attention_mask, and label separately
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': self.labels[idx]
        }

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Create datasets
train_dataset = CustomDataset(train_df, tokenizer, text_column, numerical_columns, y_label_column)
val_dataset = CustomDataset(val_df, tokenizer, text_column, numerical_columns, y_label_column)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Move model to appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Training and validation loop
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
train_precisions = []
val_precisions = []
train_recalls = []
val_recalls = []
train_f1_scores = []
val_f1_scores = []
epochs = 3
for epoch in range(epochs):
    # Training loop
    model.train()
    epoch_train_loss = 0
    correct_train_preds = 0
    total_train_preds = 0
    train_predicted_labels = []
    train_true_labels = []
    for batch in train_loader:
        # Move inputs and labels to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        # Forward pass with appropriate input names
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimizer step
        loss.backward()
        optimizer.step()

        # Track loss
        epoch_train_loss += loss.item()

        # Calculate correct predictions
        _, predicted = torch.max(outputs.logits, 1)
        correct_train_preds += (predicted == labels).sum().item()
        total_train_preds += labels.size(0)

        # Collect predicted and true labels for metrics calculation
        train_predicted_labels.extend(predicted.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())

    # Calculate training loss and metrics
    train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = correct_train_preds / total_train_preds
    train_precision = precision_score(train_true_labels, train_predicted_labels, average='weighted')
    train_recall = recall_score(train_true_labels, train_predicted_labels, average='weighted')
    train_f1 = f1_score(train_true_labels, train_predicted_labels, average='weighted')

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train F1: {train_f1:.4f}")

    # Validation loop
    model.eval()
    epoch_val_loss = 0
    correct_val_preds = 0
    total_val_preds = 0
    val_predicted_labels = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_loader:
            # Move inputs and labels to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Forward pass with appropriate input names
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Track validation loss
            epoch_val_loss += loss.item()

            # Calculate correct predictions
            _, predicted = torch.max(outputs.logits, 1)
            correct_val_preds += (predicted == labels).sum().item()
            total_val_preds += labels.size(0)

            # Collect predicted and true labels for metrics calculation
            val_predicted_labels.extend(predicted.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    # Calculate validation loss and metrics
    val_loss = epoch_val_loss / len(val_loader)
    val_accuracy = correct_val_preds / total_val_preds
    val_precision = precision_score(val_true_labels, val_predicted_labels, average='weighted')
    val_recall = recall_score(val_true_labels, val_predicted_labels, average='weighted')
    val_f1 = f1_score(val_true_labels, val_predicted_labels, average='weighted')

    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}")

# This concludes the training and validation loop.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1/3, Train Loss: 0.0450, Train Accuracy: 0.9846, Train Precision: 0.9847, Train Recall: 0.9846, Train F1: 0.9846
Val Loss: 0.0545, Val Accuracy: 0.9823, Val Precision: 0.9829, Val Recall: 0.9823, Val F1: 0.9823
Epoch 2/3, Train Loss: 0.0081, Train Accuracy: 0.9973, Train Precision: 0.9973, Train Recall: 0.9973, Train F1: 0.9973
Val Loss: 0.0667, Val Accuracy: 0.9794, Val Precision: 0.9802, Val Recall: 0.9794, Val F1: 0.9794
Epoch 3/3, Train Loss: 0.0065, Train Accuracy: 0.9979, Train Precision: 0.9979, Train Recall: 0.9979, Train F1: 0.9979
Val Loss: 0.0347, Val Accuracy: 0.9888, Val Precision: 0.9890, Val Recall: 0.9888, Val F1: 0.9888


In [17]:
import torch

# Assuming model is your BERT model instance
model_state_dict = model.state_dict()
torch.save(model_state_dict, 'BERT_MODELHC3File.pth')


In [18]:
from transformers import BertConfig, BertForSequenceClassification

# Load the configuration
config = BertConfig.from_pretrained('bert-base-uncased')

# Create an instance of the model architecture
model = BertForSequenceClassification(config)

# Load the saved model state dictionary
model_state_dict = torch.load('BERT_MODELHC3File.pth')

# Load the model state dictionary
model.load_state_dict(model_state_dict)


<All keys matched successfully>

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
# Load test dataset
df = pd.read_csv("/home/yadagiri/CLEANING DATA PAN 24 TRAIN AND TEST/test_dataset1.csv")

# Define columns
text_column = "text"
numerical_columns = ['Vocabulary', 'Noun Count', 'Verb Count', 'AUX Count', 'NUM Count', 'PRON Count', 'ADV Count', 'INTJ Count', 'PART Count']
y_label_column = "label"

# Create DataLoader for test set
batch_size = 32
test_dataset = CustomDataset(df, tokenizer, text_column, numerical_columns, y_label_column)
#test_dataset = CustomDataset(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#test_dataset = CustomDataset(test_df['text'], test_df['label'], tokenizer)  # Assuming 'binary_label' is the name of the label column
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Initialize empty lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Evaluate model on test set
model.eval()
test_loss = 0
correct_preds = 0
total_preds = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate average test loss and accuracy
avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct_preds / total_preds

# Calculate precision, recall, and F1 score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print metrics
print(f"Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy}")
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot confusion matrix
def plot_confusion_matrix(true_labels, predicted_labels, labels):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Plot confusion matrix for test set
y_true = df['label']
y_pred = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)
        y_pred.extend(predicted.cpu().numpy())
class_names = ['Human', 'AI']  # Rename '0' to 'Human' and '1' to 'AI'
plot_confusion_matrix(y_true, y_pred, labels=class_names)
# plot_confusion_matrix(y_true, y_pred, labels=['0', '1'])  # Replace labels with actual class labels if different

# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()
